<a href="https://colab.research.google.com/github/nendter/A-sl-bc/blob/main/notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Load dataset into environment

!pip install kiggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download grassknoted/asl-alphabet
!unzip -o asl-alphabet.zip -d /content/asl-ds/raw

In [29]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf

In [ ]:
### Load dataset

def load_dataset(raw_folder_path):

  # Train
  train_raw_folder_path = os.path.join(raw_folder_path, "asl_alphabet_train/asl_alphabet_train/")
  train_imgs = []
  train_lbls = []

  for i, lbl in enumerate(os.listdir(train_raw_folder_path)):
    lbl_train_raw_folder_path = os.path.join(train_raw_folder_path, lbl)

    for j, img_name in enumerate(os.listdir(lbl_train_raw_folder_path)):
      img_train_raw_folder_path = os.path.join(lbl_train_raw_folder_path, img_name)
      img = Image.open(img_train_raw_folder_path)
      img = np.array(img) / 255
      train_imgs.append(img)
      train_lbls.append(lbl)

  # Test
  test_raw_folder_path = os.path.join(raw_folder_path, "asl_alphabet_test/asl_alphabet_test/")
  test_imgs = []
  test_lbls = []

  for i, lbl in enumerate(os.listdir(test_raw_folder_path)):
    lbl_test_raw_folder_path = os.path.join(test_raw_folder_path, lbl)

    for j, img_name in enumerate(os.listdir(lbl_test_raw_folder_path)):
      img_test_raw_folder_path = os.path.join(lbl_test_raw_folder_path, img_name)
      img = Image.open(img_test_raw_folder_path)
      img = np.array(img) / 255
      test_imgs.append(img)
      test_lbls.append(lbl)

  return train_imgs, train_lbls, test_imgs, test_lbls

train_imgs, train_lbls, test_imgs, test_lbls = load_dataset("/content/asl-ds/raw")

print(train_imgs.size())